# Exercício - Titanic
Este é um código junto com o famoso conjunto de dados titânico, é sempre bom começar com este conjunto de dados porque é um exemplo que você encontrará em praticamente todas as linguagens de análise de dados.

In [24]:
from pyspark.sql import SparkSession

In [25]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [26]:
data = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [27]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [28]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [29]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [30]:
my_final_data = my_cols.na.drop()

### Trabalhando com colunas categóricas

Vamos dividir isso em várias etapas para deixar tudo claro.

In [31]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [32]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [33]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [34]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [35]:
from pyspark.ml.classification import LogisticRegression

## Pipelines

Vamos ver um exemplo de como usar pipelines

In [36]:
from pyspark.ml import Pipeline

In [37]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [38]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [39]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [40]:
fit_model = pipeline.fit(train_titanic_data)

In [41]:
results = fit_model.transform(test_titanic_data)

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [44]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [48]:
AUC = my_eval.evaluate(results)

In [55]:
AUC = my_eval.evaluate(results)

In [57]:
AUC

0.7809178743961354